<a href="https://colab.research.google.com/github/maybje/CE888/blob/master/lab5/lab5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Lab 5  - Recommender systems

In [0]:
import pandas as pd
import numpy as np

### 1. Load the data from ``jester-data-1.csv''

In [2]:
rates = pd.read_csv("https://raw.githubusercontent.com/albanda/CE888/master/lab5-recommender/jester-data-1.csv", header=None)
columns=rates.columns
rates=rates[columns[1:]]
rates.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,-7.82,8.79,-9.66,-8.16,-7.52,-8.50,-9.85,4.17,-8.98,-4.76,-8.50,-6.75,-7.18,8.45,-7.18,-7.52,-7.43,-9.81,-9.85,-9.85,-9.37,1.50,-4.37,-9.81,-8.50,1.12,7.82,2.86,9.13,-7.43,2.14,-4.08,-9.08,7.82,5.05,4.95,-9.17,-8.40,-8.40,-8.40,...,8.59,3.59,-6.84,-9.03,2.82,-1.36,-9.08,8.30,5.68,-4.81,99.0,99.00,99.0,99.00,99.00,99.00,99.00,-9.42,99.00,99.00,99.00,-7.72,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,2.82,99.00,99.00,99.00,99.00,99.00,-5.63,99.00,99.00,99.00
1,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,8.88,9.22,6.75,8.64,4.42,7.43,4.56,-0.97,4.66,-0.68,3.30,-1.21,0.87,8.64,8.35,9.17,0.05,7.57,4.71,0.87,-0.39,6.99,6.50,-0.92,7.14,9.03,-1.80,0.73,7.09,3.40,-0.87,7.91,...,-6.70,-3.35,-9.03,4.47,4.08,-3.83,8.74,1.12,0.78,7.52,-5.0,2.77,8.3,7.77,7.33,6.21,7.72,8.98,8.64,8.20,3.93,4.85,4.85,6.07,8.98,4.51,-0.05,3.69,4.56,0.58,2.82,-4.95,-0.29,7.86,-0.19,-2.14,3.06,0.34,-4.32,1.07
2,99.00,99.00,99.00,99.00,9.03,9.27,9.03,9.27,99.00,99.00,7.33,7.57,9.37,6.17,-6.36,-6.89,-7.86,9.03,9.03,9.03,7.28,99.00,8.25,99.00,99.00,7.48,7.28,7.28,8.93,99.00,6.17,7.28,99.00,99.00,8.98,7.33,99.00,6.17,9.08,7.33,...,6.46,7.28,99.00,99.00,7.04,7.28,99.00,7.28,8.25,99.00,99.0,99.00,99.0,99.00,99.00,8.93,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,9.03,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00
3,99.00,8.35,99.00,99.00,1.80,8.16,-2.82,6.21,99.00,1.84,7.33,6.60,6.31,8.11,-7.23,-6.65,1.17,-6.60,-3.64,-2.09,5.34,99.00,99.00,99.00,99.00,2.91,3.93,6.75,6.60,99.00,6.65,-6.12,99.00,7.57,6.21,6.65,99.00,-8.30,7.18,2.82,...,0.00,-3.69,99.00,99.00,7.82,0.24,99.00,7.28,-2.33,99.00,99.0,99.00,99.0,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,0.63,99.00,99.00,-2.33,99.00,99.00,99.00,99.00,99.00,0.53,99.00,99.00,99.00,99.00,99.00,99.00
4,8.50,4.61,-4.17,-5.39,1.36,1.60,7.04,4.61,-0.44,5.73,8.25,6.84,-3.93,7.23,-2.33,-9.66,2.72,-1.36,2.57,4.51,8.20,6.12,8.30,-1.26,7.77,1.89,-1.17,5.68,8.45,4.61,8.06,-9.47,7.28,5.68,2.48,3.20,-1.26,6.80,4.51,2.48,...,7.38,6.17,4.71,-2.28,7.38,4.56,7.14,4.22,3.01,3.83,99.0,99.00,99.0,99.00,99.00,99.00,4.13,99.00,99.00,99.00,5.24,5.92,0.87,7.28,3.93,-0.63,6.31,4.71,2.82,2.96,5.19,5.58,4.27,5.19,5.73,1.55,3.11,6.55,1.80,1.60


### 2.  Using the helper function provided in the helper_function notebook (or create your own), label 10% of the dataset cells as 99. This is your validation set. Keep the the actual values of the cells so you can use them later.

In [0]:
def replace(orig, percentage=0.1):
  """
  Replaces 'percentage'% of the original values in 'orig' with 99's
  :param orig: original data array
  :param percentage: percentage of values to replace (0<percentage<1)
  """
  new_data = orig.copy()
  rated = np.where(arr!=99)
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])

In [0]:
arr=rates.values
new_arr, idx = replace(arr, 0.1)

### 3. Use latent factor modeling to infer the hidden ratings of the users (they are labeled as "99" in the dataset) on the training set

In [0]:
n_latent_factors = 2

user_ratings = new_arr
# Initialise as random values
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))

In [0]:
def predict_rating(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)


def train(user_id, item_id, rating, alpha=0.0001):
    
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  prediction_rating - rating
    #print err
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    

def sgd(iterations=3000, n_print=500, alfa=0.0001):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(iterations):
        error = []
        mse=float("inf")
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = user_ratings[user_id][item_id]
                if rating!=99:
                    err = train(user_id, item_id, rating,alfa)
                    error.append(err)
        mse_n = (np.array(error) ** 2).mean()
        if np.isnan(mse_n):
          break   
        elif mse_n>mse:
          print("Started to diverge: ",  mse)
          break
        else:
          mse=mse_n
        if (iteration % n_print) == 0:
            print(mse)

In [0]:
#It starts converging after 100 iterations, and seems to start diverging afterwards.
sgd(iterations=201,n_print=50)

26.265050992613425
17.1638053378769
17.081100939505095
17.082869733174423


### 4.  Calculate the performance (e.g., MSE) of the algorithm on the validation dataset

In [0]:
predictions = latent_user_preferences.dot(latent_item_features.T)

In [0]:
errors =(predictions[idx[0][:], idx[1][:]] - arr[idx[0][:], idx[1][:]])**2

mse=sum(errors)/len(errors)
mse

18.094688017266957

## 5. Repeat the two points above changing hyper-parameters (i.e., learning rate, number of iterations of SVD, number of latent factors, etc.) as needed to get good results (you can create multiple validation sets if you want, and run a bootstrap!)

### Grid Search
It was noticed learning rates greater than 0.0001 produce 'nan' values. For this reason, we just loop through the number of factors and increase the number of iterations. We added a condition to the "sgd" function: if it starts diverging it will stop.

In [0]:
arr=rates.values
new_arr, idx = replace(arr, 0.1)

In [0]:
n_latent_factors = [2,3,4,5,6]
alph=[.0001]
mse_i=float("inf")

In [0]:
for l in n_latent_factors:
  for a in alph:
    print("Testing: ", l, a)

    user_ratings = new_arr
    # Initialise as random values
    latent_user_preferences = np.random.random((user_ratings.shape[0], l))
    latent_item_features = np.random.random((user_ratings.shape[1], l))

    sgd(iterations=201,n_print=50, alfa=a)
    predictions = latent_user_preferences.dot(latent_item_features.T)
    errors =(predictions[idx[0][:], idx[1][:]] - arr[idx[0][:], idx[1][:]])**2
    n_mse=sum(errors)/len(errors) 
    
    print("Current combination: ", n_mse)
    if n_mse<mse_i:
      mse_i=n_mse
      num_l=l
      opt_alph=a

print("Optimal number of factors: ", num_l)    
print("Optimal alpha: ", opt_alph)
print("min mse: ", mse_i)

Testing:  2 0.0001
26.2289314115267
17.22568891419005
17.08540149496422
17.086883800650906
17.089577717742227
Current combination:  18.040001573385606
Testing:  3 0.0001
25.891895863178803
17.17912864201001
16.307899025134436
16.08145077919943
16.06776461266531
Current combination:  17.51511109964257
Testing:  4 0.0001
25.74783366193592
17.084218756420398
15.654460618706118
15.324207416193895
15.252098403467022
Current combination:  17.209268641657136
Testing:  5 0.0001
25.606965109546092
17.17844327650152
15.309475056357604
14.622783751702782
14.537273014535987
Current combination:  16.978066576191654
Testing:  6 0.0001
25.550860429596277
17.27405214291506
15.325716661379381
14.335064420918886
14.05865198829248
Current combination:  16.958645896796998
Optimal number of factors:  6
Optimal alpha:  0.0001
min mse:  16.958645896796998


It was noticed the more latent factors, the smaller the errror becomes. We decide to train our model with 6 latent factors and train it for more iterations.

In [7]:
user_ratings = new_arr
# Initialise as random values
latent_user_preferences = np.random.random((user_ratings.shape[0], 6))
latent_item_features = np.random.random((user_ratings.shape[1], 6))

sgd(iterations=501,n_print=50, alfa=0.0001)
predictions = latent_user_preferences.dot(latent_item_features.T)
errors =(predictions[idx[0][:], idx[1][:]] - arr[idx[0][:], idx[1][:]])**2
n_mse=sum(errors)/len(errors) 

25.55733996044708
17.20795668795721
15.174565868208227
14.23242523970631
14.067851116969832
14.013905305342192
13.995256865998877
13.990072559276522
13.990654026812969
13.994246172241905
13.999784841044445


Final MSE

In [10]:
print("test mse: ", n_mse)

test mse:  17.067276684718603


## 6. Once you're happy, make predictions for the test dataset

In [0]:
n_latent_factors = 6

user_ratings = rates.values
# Initialise as random values
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))

In [12]:
sgd(iterations=351,n_print=50, alfa=.0001)

25.53416211218568
17.063147699572905
14.904035878281421
14.299038242825118
14.184778997049117
14.163264479581931
14.159296292413604
14.160867318413326


In [0]:
predictions = latent_user_preferences.dot(latent_item_features.T)
values = [zip(user_ratings[i], predictions[i].round(2)) for i in range(predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data.columns = rates.columns

In [23]:
comparison_data

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,"(-7.82, -2.12)","(8.79, -1.75)","(-9.66, -6.87)","(-8.16, -9.92)","(-7.52, -7.15)","(-8.5, -2.87)","(-9.85, -6.76)","(4.17, -5.4)","(-8.98, -6.96)","(-4.76, -2.1)","(-8.5, -3.04)","(-6.75, -1.39)","(-7.18, -6.32)","(8.45, -0.44)","(-7.18, -6.22)","(-7.52, -10.64)","(-7.43, -6.54)","(-9.81, -5.08)","(-9.85, -6.49)","(-9.85, -9.9)","(-9.37, 0.41)","(1.5, -2.26)","(-4.37, -5.62)","(-9.81, -10.94)","(-8.5, -3.0)","(1.12, -0.46)","(7.82, 2.03)","(2.86, -2.4)","(9.13, 4.5)","(-7.43, -9.19)","(2.14, 0.02)","(-4.08, 2.83)","(-9.08, -10.21)","(7.82, -2.05)","(5.05, 3.3)","(4.95, 3.37)","(-9.17, -7.12)","(-8.4, -1.86)","(-8.4, -2.81)","(-8.4, -4.81)",...,"(8.59, -0.23)","(3.59, 2.07)","(-6.84, -4.93)","(-9.03, -8.63)","(2.82, 1.54)","(-1.36, 1.05)","(-9.08, -7.48)","(8.3, 1.33)","(5.68, 2.41)","(-4.81, -4.58)","(99.0, -7.24)","(99.0, 0.67)","(99.0, -7.44)","(99.0, -11.14)","(99.0, -9.2)","(99.0, -0.06)","(99.0, -7.1)","(-9.42, -5.3)","(99.0, -8.74)","(99.0, -6.34)","(99.0, -1.08)","(-7.72, -5.87)","(99.0, -1.18)","(99.0, -5.98)","(99.0, -6.93)","(99.0, -5.31)","(99.0, -1.05)","(99.0, -0.67)","(99.0, 3.16)","(99.0, -7.0)","(2.82, -0.54)","(99.0, -2.78)","(99.0, 0.49)","(99.0, -3.26)","(99.0, -3.05)","(99.0, -1.63)","(-5.63, -0.73)","(99.0, -2.16)","(99.0, -6.45)","(99.0, -4.33)"
1,"(4.08, 3.28)","(-0.29, 1.91)","(6.36, 6.96)","(4.37, 6.27)","(-2.38, 3.51)","(-9.66, 4.57)","(-0.73, -1.52)","(-5.34, 0.37)","(8.88, 6.2)","(9.22, 4.55)","(6.75, 5.03)","(8.64, 3.51)","(4.42, -0.21)","(7.43, 2.47)","(4.56, -1.48)","(-0.97, 2.3)","(4.66, 1.61)","(-0.68, -0.28)","(3.3, 0.82)","(-1.21, 2.3)","(0.87, 2.12)","(8.64, 3.83)","(8.35, 4.85)","(9.17, 8.62)","(0.05, 3.11)","(7.57, 2.18)","(4.71, 1.75)","(0.87, 1.73)","(-0.39, -2.07)","(6.99, 6.85)","(6.5, 2.93)","(-0.92, -0.82)","(7.14, 6.14)","(9.03, 4.66)","(-1.8, -0.77)","(0.73, 0.07)","(7.09, 2.58)","(3.4, 2.85)","(-0.87, 3.11)","(7.91, 4.18)",...,"(-6.7, 2.7)","(-3.35, -0.08)","(-9.03, 1.32)","(4.47, 5.58)","(4.08, 2.46)","(-3.83, 1.3)","(8.74, 6.92)","(1.12, 0.05)","(0.78, 0.28)","(7.52, 5.11)","(-5.0, -0.7)","(2.77, 0.96)","(8.3, 5.84)","(7.77, 4.86)","(7.33, 5.83)","(6.21, 1.63)","(7.72, 4.41)","(8.98, 4.29)","(8.64, 6.15)","(8.2, 5.96)","(3.93, 2.41)","(4.85, 1.82)","(4.85, 1.61)","(6.07, 1.02)","(8.98, 4.13)","(4.51, 2.48)","(-0.05, 2.11)","(3.69, 0.83)","(4.56, -1.06)","(0.58, 1.43)","(2.82, 1.15)","(-4.95, 2.03)","(-0.29, 0.26)","(7.86, 2.13)","(-0.19, 1.38)","(-2.14, 0.79)","(3.06, -0.67)","(0.34, 1.65)","(-4.32, 2.18)","(1.07, 4.56)"
2,"(99.0, 7.23)","(99.0, 5.51)","(99.0, 8.04)","(99.0, 5.29)","(9.03, 6.78)","(9.27, 8.38)","(9.03, 5.86)","(9.27, 4.57)","(99.0, 7.16)","(99.0, 7.92)","(7.33, 8.71)","(7.57, 7.89)","(9.37, 2.21)","(6.17, 6.98)","(-6.36, 2.4)","(-6.89, 2.33)","(-7.86, 2.58)","(9.03, 5.19)","(9.03, 6.95)","(9.03, 6.56)","(7.28, 7.14)","(99.0, 7.03)","(8.25, 7.85)","(99.0, 6.13)","(99.0, 6.25)","(7.48, 6.56)","(7.28, 8.43)","(7.28, 8.87)","(8.93, 7.06)","(99.0, 7.86)","(6.17, 7.85)","(7.28, 8.71)","(99.0, 6.38)","(99.0, 6.61)","(8.98, 8.2)","(7.33, 8.17)","(99.0, 5.24)","(6.17, 8.31)","(9.08, 6.93)","(7.33, 8.45)",...,"(6.46, 8.95)","(7.28, 9.34)","(99.0, 7.86)","(99.0, 7.5)","(7.04, 7.01)","(7.28, 8.61)","(99.0, 6.36)","(7.28, 8.76)","(8.25, 7.96)","(99.0, 7.28)","(99.0, 5.08)","(99.0, 7.68)","(99.0, 7.9)","(99.0, 5.22)","(99.0, 6.11)","(8.93, 7.47)","(99.0, 7.41)","(99.0, 8.56)","(99.0, 7.01)","(9.08, 7.81)","(99.0, 6.88)","(99.0, 8.1)","(99.0, 6.83)","(99.0, 7.82)","(99.0, 7.62)","(99.0, 6.99)","(99.0, 7.16)","(99.0, 7.98)","(99.0, 8.57)","(9.03, 8.2)","(99.0, 7.79)","(99.0, 7.02)","(99.0, 7.94)","(9.08, 7.77)","(99.0, 6.99)","(99.0, 7.3)","(99.0, 7.13)","(99.0, 4.75)","(99.0, 6.34)","(99.0, 6.8)"
3,"(99.0, 2.5)","(8.35, 5.55)","(99.0, 4.08)

Some of the predictions are quite close to the real value, however some others are very far. Increasing the number of latent factors could help to reduce the error. 

On the other hand, increasing the number of iterations helps up to one point when the error starts converging to a steady number, and even diverging. The soultion is increasing even more the number of latent factors (not explored due to the running time (several hours)).